# Data summarizing code

Save the code snipets that summarize data obtained directly from batch processes such as *df2_kinetics.py*. The snipets here should generate a *summary.csv* file for each day's data (of a specific type). 

## Guidelines
- Each snipet should contain a summarizing part and a retrieval part

## <font color='blue'> Plans

<font color='blue'>
- ...
    
</font>

## Table of contents

1. Correlation length: angle and velocity
2. df2_kinetics
    - GNF curves at different concentrations
    - $\alpha$
    - Order parameter
3. Summarize local correlation between df and energy
4. Summarize energy spectrum data

## 0 Packages

In [7]:
import pandas as pd
import numpy as np
import os
from corrLib import readdata
from corr_utils import *
from IPython.display import clear_output

In [20]:
data_master_dir = r'E:\Google Drive' # data folder: Google drive for now, but may change in the future
dirs = data_log_mapping(kw='aug') # video info for GNF raw data, obtained from Aug 3rd to Aug 6th

### 1 Correlation length: angle and velocity

In [ ]:
# summarize data
# sample 10 frames out of the last 1/3 of frames
corr_data_path = r'E:\moreData\08052020\cav_imseq'
num_sample = 20 # number of frames to sample
sample_clA = []
sample_clV = []
sfL = next(os.walk(corr_data_path))[1]
sfL_new = []
for s in sfL:
#     print('sample ' + str(n))
    corr_data_path_num = os.path.join(corr_data_path, s)
    
    # sample 20 frames in the last 1/3 frames
    
    l = len(corrLib.readdata(corr_data_path_num))
    if l > 60:
        samples = np.random.randint((l*2/3)/2, (l-1)/2, num_sample) * 2
        clAL = []
        clVL = []
        for i in samples:
            data_raw = pd.read_csv(os.path.join(corr_data_path_num, '{0:04d}-{1:04d}.csv'.format(i, i+1))) # X Y CA CV
            data_AV = xy_to_r(data_raw).sort_values(by=['R'])     
            clA, fit = corr_length(data_AV.rename(columns={'CA': 'C'}), fitting_range=500)
            clV, fit = corr_length(data_AV.rename(columns={'CV': 'C'}), fitting_range=500)
            clAL.append(clA)
            clVL.append(clV)
        sample_clA.append(np.array(clAL).mean())
        sample_clV.append(np.array(clVL).mean())
        sfL_new.append(int(s))
    else:
        raise ValueError('Too few data to sample from. Need at least 60 .csv files.')
summary = pd.DataFrame({'sample': sfL_new, 'clA': sample_clA, 'clV': sample_clV})
summary.to_csv(os.path.join(corr_data_path, 'summary.csv'), index=False)
### Needs further improvement

In [ ]:
# retrieve data from cav_imseq/summary.csv
folder = r'E:\moreData'
subfolder_name = 'cav_imseq'
data = {'conc': [], 'clA_avg': [], 'clV_avg': [], 'clA_std': [], 'clV_std': []}
for kw in dirs:
    if kw != '00':
        conc = int(kw)
        for n, dn in enumerate(dirs[kw]):
            date, num = dn.split('-')
            summary_dir = os.path.join(folder, date, subfolder_name, 'summary.csv')
            temp = pd.read_csv(summary_dir, index_col='sample').loc[[int(num)]]
            if n == 0:
                conc_data = temp
            else:
                conc_data = conc_data.append(temp)
        data['conc'].append(conc)
        data['clA_avg'].append(conc_data.clA.mean())
        data['clV_avg'].append(conc_data.clV.mean())
        data['clA_std'].append(conc_data.clA.std())
        data['clV_std'].append(conc_data.clV.std())

## 2 df2_kinetics

### 2.1 GNF curves at different concentrations

In [21]:
date = []
num = []
conc = []
for kw in dirs:
    if int(kw) <= 120:
        conc.append(int(kw))
        d, n = dirs[kw][1].split('-')
        date.append(d)
        num.append(int(n))
data_log = pd.DataFrame().assign(conc=conc, date=date, num=num).sort_values(by='conc')
# NOTE: 
# This block is not elegant. I want to use a consistent method to load experiment log information and map them to data.
# Throughout this project, I have implemented no less than three of such methods. Yet, none of them are intuitive or easy to use.
# Keep this block here for now, change it in the future.

In [22]:
lb = 10
data_dir = r'E:\moreData\{0}\df2_kinetics\{1:02d}\kinetics_data.csv'
count = 0
for num, i in data_log.iterrows():
    if i.conc != 0 and i.conc != 85:# and i.conc < 40:     # for animation in slides   
        k_data = pd.read_csv(data_dir.format(i.date, i.num))
        gnf_data = k_data.loc[k_data.segment==k_data.segment.max()]
        x, y = postprocess_gnf(gnf_data, lb, xlim=[10, 10000], sparse=1)
#         ax.plot(x, y, marker=marker_list[color_dict[str(i.conc)]], label='{:.1f}'.format(i.conc*0.08), color=wowcolor(color_dict[str(i.conc)]), markersize=2, lw=1)
        temp = pd.concat([x, y], axis=1, keys=['N', str(i.conc)]).set_index('N')
        if count == 0:
            master = temp
        else:
            master = pd.concat([master, temp], axis=1)
        count += 1
master.to_csv(os.path.join(data_master_dir, r'Research projects\DF\data\gnf.csv'))

In [17]:
def postprocess_gnf(gnf_data, lb, xlim=None, sparse=3):
    """
    Postprocess raw GNF data for plotting.
    
    Args:
    gnf_data -- DataFrame containing columns ('n', 'd'), generated by df2_nobp.py
    lb -- size of bacteria (pixel, normalizing factor of x axis)
    xlim -- box size beyond which the data get cut off (pixel), can be either integer or a list of 2 integers
            if xlim is int, it is the upper limit, data above xlim will be cut off,
            if xlim is a list, data outside [xlim[0], xlim[1]] will be cut off
    sparse -- the degree to sparsify the data, 1 is doing nothing, 3 means only keep 1/3 of the orginal data
    
    Returns:
    x, y -- a tuple that can be plotted directly using plt.plot(x, y)
    """    
    
    if xlim == None:
        data = gnf_data
    elif isinstance(xlim, int):
        data = gnf_data.loc[gnf_data.n < xlim*lb**2]
    elif isinstance(xlim, list) and len(xlim) == 2:
        data = gnf_data.loc[(gnf_data.n>=xlim[0]*lb**2)&(gnf_data.n < xlim[1]*lb**2)]
        
    xx = data.n / lb**2
    yy = data.d / data.n**0.5
    
    # sparcify
    x = xx[0:len(xx):sparse]
    y = yy[0:len(xx):sparse]
    
    return x, y

### 2.2 $\alpha$

In [ ]:
# Summarize data from each raw data folders
dates = ['08032020', '08042020', '08052020', '08062020']
sampling_range = 1 # range between 0 and 1, above which we sample the alpha values and average
fitting_range = [10, ] # um
############################################

for date in dates:
    date_folder = r'D:\density_fluctuations\{}'.format(date)
    k_master_folder = os.path.join(date_folder, 'df2_kinetics')
    i_master_folder = os.path.join(date_folder, 'overall_intensity')
    sfL = next(os.walk(k_master_folder))[1]
    alpha_list = []
    n_list = []
    for sf in sfL:
        n = int(sf)
        fps = data_log()[date]['fps'][n]
        k_folder = r'D:\density_fluctuations\{0}\df2_kinetics\{1:02d}'.format(date, n)
        i_folder = r'D:\density_fluctuations\{0}\overall_intensity\{1:02d}'.format(date, n)
        k_data = pd.read_csv(os.path.join(k_folder, 'kinetics_data.csv'))
        i_data = pd.read_csv(os.path.join(i_folder, 'intensity.csv'))
        data = plot_kinetics(k_data, i_data, tlim=None, xlim=[5, 100], lb=10, mpp=0.33, seg_length=50, fps=fps, plot=False)
        df = pd.DataFrame().assign(t=data['t0'], alpha=data['alpha'])
        alpha = df['alpha'].loc[df['t']>=df['t'].max()*sampling_range].mean()
        n_list.append(n)
        alpha_list.append(alpha)
    data = pd.DataFrame({'n': n_list, 'alpha': alpha_list})
    data.to_csv(os.path.join(k_master_folder, 'summary.csv'), index=False)

### 2.3 Order parameter

In [ ]:
eo_folder = r'D:\density_fluctuations\08042020\energy_order'
sfL = next(os.walk(eo_folder))[1]
OP_list = []
E_list = []
for sf in sfL:
    print('summarizing data in {}'.format(sf))
    eo_data = pd.read_csv(os.path.join(eo_folder, sf, 'energy_order.csv'))
    eo_data_crop = eo_data.loc[eo_data.t>eo_data.t.max()*0.9]
    E = eo_data_crop.E.mean()
    E_list.append(E)
    OP = eo_data_crop.OP.mean()
    OP_list.append(OP)

In [ ]:
data = pd.DataFrame().assign(sample=sfL, E=E_list, OP=OP_list)
data.to_csv(os.path.join(eo_folder, 'eo_summary.csv'), index=False)

## 3 summarize local correlation between df and energy

In [ ]:
folder = r'D:\density_fluctuations\08062020\local_df'
sfL = next(os.walk(folder))[1]
avg_list = []
for sf in sfL:
    print('summarizing data in {}'.format(sf))
    df_folder = os.path.join(folder, sf, 'dt=10')
    piv_folder = r'D:\density_fluctuations\08062020\piv_imseq\{}'.format(sf)
    l = readdata(df_folder, 'npy')
    l = l.loc[l.Name.astype('int')>l.Name.astype('int').max()*0.8]
    corr_list = []
    for num, i in l.iterrows():
        df = np.load(i.Dir)
        n = int(i.Name)
        pivData = pd.read_csv(os.path.join(piv_folder, '{0:04d}-{1:04d}.csv'.format(n, n+1)))
        v = np.array(pivData.u**2 + pivData.v**2).reshape((42, 50))
        corr = corr2d(df, v)
        corr_list.append(corr)
    avg_corr = np.array(corr_list).mean()
    avg_list.append(avg_corr)
    print('{0}: {1:.3f}'.format(sf, avg_corr))

In [ ]:
data = pd.DataFrame().assign(sample=sfL, E=avg_list)
data.to_csv(os.path.join(folder, 'corr_E_sl=10.csv'), index=False)

## 4 Summarize energy spectrum data

In [24]:
def summarize_es(folder):
    """
    Average energy spectrum data in each subfolder under folder, then save a summary.csv file under folder, containing all the averaged data.
    
    """
    sfL = next(os.walk(folder))[1]
    count = 0
    for sf in sfL:
        clear_output(wait=True)
        print('Summarizing {0}\\{1} ...'.format(folder, sf))
        es_folder = os.path.join(folder, sf) # for varstep, experiment number '00' needs to be included
        l = readdata(es_folder, 'csv')
        for num, i in l.iterrows():
            if num == 0:
                data = pd.read_csv(i.Dir)
            else:
                data += pd.read_csv(i.Dir)
            print(data)
        data /= num + 1
        if count == 0:
            summary = data.assign(sample=sf)
        else:
            summary = summary.append(data.assign(sample=sf))
        count += 1
    summary.to_csv(os.path.join(folder, 'summary.csv'), index=False)

In [25]:
folders = ['08032020']#, '08042020', '08052020', '08062020']
for f in folders:
    folder = os.path.join(r'E:\moreData', f, 'energy_spectrum')
    summarize_es(folder)

Summarizing E:\moreData\08032020\energy_spectrum\08 ...
                 E         k
0     62288.014188  0.000000
1     26214.875225  0.002410
2     26214.875225  0.002410
3     33803.529487  0.002869
4     33803.529487  0.002869
...            ...       ...
2095      1.003689  0.083190
2096      1.003689  0.083190
2097      1.912703  0.083507
2098      1.912703  0.083507
2099      8.226007  0.085194

[2100 rows x 2 columns]
                  E         k
0     110817.427050  0.000000
1      38609.161854  0.004819
2      38609.161854  0.004819
3      92510.854522  0.005737
4      92510.854522  0.005737
...             ...       ...
2095       3.966402  0.166380
2096       3.966402  0.166380
2097       9.112242  0.167014
2098       9.112242  0.167014
2099      13.407658  0.170387

[2100 rows x 2 columns]
                  E         k
0     225620.605336  0.000000
1      66584.083205  0.007229
2      66584.083205  0.007229
3     151185.654430  0.008606
4     151185.654430  0.008606
...   

[2100 rows x 2 columns]
                 E         k
0     2.665883e+06  0.000000
1     2.092627e+06  0.060241
2     2.092627e+06  0.060241
3     2.225862e+06  0.071715
4     2.225862e+06  0.071715
...            ...       ...
2095  1.219652e+02  2.079748
2096  1.219652e+02  2.079748
2097  7.636543e+01  2.087678
2098  7.636543e+01  2.087678
2099  1.071048e+02  2.129840

[2100 rows x 2 columns]
                 E         k
0     2.814900e+06  0.000000
1     2.184332e+06  0.062651
2     2.184332e+06  0.062651
3     2.337191e+06  0.074584
4     2.337191e+06  0.074584
...            ...       ...
2095  1.307221e+02  2.162937
2096  1.307221e+02  2.162937
2097  7.858259e+01  2.171185
2098  7.858259e+01  2.171185
2099  1.182848e+02  2.215033

[2100 rows x 2 columns]
                 E         k
0     2.987056e+06  0.000000
1     2.275869e+06  0.065060
2     2.275869e+06  0.065060
3     2.419791e+06  0.077453
4     2.419791e+06  0.077453
...            ...       ...
2095  1.345385e+02  2.24612

                 E         k
0     5.590407e+06  0.000000
1     4.418342e+06  0.125301
2     4.418342e+06  0.125301
3     4.051340e+06  0.149168
4     4.051340e+06  0.149168
...            ...       ...
2095  2.276053e+02  4.325875
2096  2.276053e+02  4.325875
2097  1.887259e+02  4.342369
2098  1.887259e+02  4.342369
2099  2.253899e+02  4.430067

[2100 rows x 2 columns]
                 E         k
0     5.615920e+06  0.000000
1     4.494478e+06  0.127711
2     4.494478e+06  0.127711
3     4.099546e+06  0.152037
4     4.099546e+06  0.152037
...            ...       ...
2095  2.343778e+02  4.409065
2096  2.343778e+02  4.409065
2097  1.917889e+02  4.425876
2098  1.917889e+02  4.425876
2099  2.352018e+02  4.515260

[2100 rows x 2 columns]
                 E         k
0     5.695642e+06  0.000000
1     4.598664e+06  0.130120
2     4.598664e+06  0.130120
3     4.147232e+06  0.154905
4     4.147232e+06  0.154905
...            ...       ...
2095  2.365274e+02  4.492255
2096  2.365274e+02  4.

[2100 rows x 2 columns]
                 E         k
0     7.294419e+06  0.000000
1     6.404521e+06  0.192771
2     6.404521e+06  0.192771
3     5.125837e+06  0.229489
4     5.125837e+06  0.229489
...            ...       ...
2095  3.497994e+02  6.655192
2096  3.497994e+02  6.655192
2097  2.940645e+02  6.680568
2098  2.940645e+02  6.680568
2099  3.436935e+02  6.815487

[2100 rows x 2 columns]
                 E         k
0     7.302900e+06  0.000000
1     6.416113e+06  0.195181
2     6.416113e+06  0.195181
3     5.256895e+06  0.232358
4     5.256895e+06  0.232358
...            ...       ...
2095  3.521687e+02  6.738382
2096  3.521687e+02  6.738382
2097  2.968392e+02  6.764075
2098  2.968392e+02  6.764075
2099  3.463970e+02  6.900681

[2100 rows x 2 columns]
                 E         k
0     7.350705e+06  0.000000
1     6.425632e+06  0.197590
2     6.425632e+06  0.197590
3     5.414125e+06  0.235227
4     5.414125e+06  0.235227
...            ...       ...
2095  3.552018e+02  6.82157

                 E         k
0     9.014698e+06  0.000000
1     7.606784e+06  0.257831
2     7.606784e+06  0.257831
3     6.824726e+06  0.306942
4     6.824726e+06  0.306942
...            ...       ...
2095  4.569842e+02  8.901320
2096  4.569842e+02  8.901320
2097  3.992560e+02  8.935260
2098  3.992560e+02  8.935260
2099  4.577045e+02  9.115714

[2100 rows x 2 columns]
                 E         k
0     9.117255e+06  0.000000
1     7.654993e+06  0.260241
2     7.654993e+06  0.260241
3     6.848067e+06  0.309811
4     6.848067e+06  0.309811
...            ...       ...
2095  4.617376e+02  8.984509
2096  4.617376e+02  8.984509
2097  4.114439e+02  9.018767
2098  4.114439e+02  9.018767
2099  4.655031e+02  9.200908

[2100 rows x 2 columns]
                 E         k
0     9.250450e+06  0.000000
1     7.700964e+06  0.262651
2     7.700964e+06  0.262651
3     6.875090e+06  0.312679
4     6.875090e+06  0.312679
...            ...       ...
2095  4.692288e+02  9.067699
2096  4.692288e+02  9.

...

[2100 rows x 2 columns]
                 E          k
0     1.561743e+09   0.000000
1     1.067895e+09   0.972107
2     1.067895e+09   0.972107
3     8.498812e+08   1.157270
4     8.498812e+08   1.157270
...            ...        ...
2095  1.509190e+05  33.560823
2096  1.509190e+05  33.560823
2097  1.520850e+05  33.688789
2098  1.520850e+05  33.688789
2099  1.262057e+05  34.369159

[2100 rows x 2 columns]
                 E          k
0     1.571329e+09   0.000000
1     1.080492e+09   0.974531
2     1.080492e+09   0.974531
3     8.539185e+08   1.160156
4     8.539185e+08   1.160156
...            ...        ...
2095  1.525084e+05  33.644517
2096  1.525084e+05  33.644517
2097  1.524536e+05  33.772802
2098  1.524536e+05  33.772802
2099  1.295010e+05  34.454869

[2100 rows x 2 columns]
                 E          k
0     1.573693e+09   0.000000
1     1.084721e+09   0.976955
2     1.084721e+09   0.976955
3     8.593602e+08   1.163042
4     8.593602e+08   1.163042
...            ...        

                 E          k
0     1.722176e+09   0.000000
1     1.322405e+09   1.032698
2     1.322405e+09   1.032698
3     9.147113e+08   1.229402
4     9.147113e+08   1.229402
...            ...        ...
2095  1.967163e+05  35.652671
2096  1.967163e+05  35.652671
2097  1.928348e+05  35.788613
2098  1.928348e+05  35.788613
2099  1.684717e+05  36.511390

[2100 rows x 2 columns]
                 E          k
0     1.749170e+09   0.000000
1     1.344413e+09   1.035122
2     1.344413e+09   1.035122
3     9.183205e+08   1.232288
4     9.183205e+08   1.232288
...            ...        ...
2095  2.004694e+05  35.736365
2096  2.004694e+05  35.736365
2097  1.941598e+05  35.872626
2098  1.941598e+05  35.872626
2099  1.690958e+05  36.597100

[2100 rows x 2 columns]
                 E          k
0     1.759452e+09   0.000000
1     1.364783e+09   1.037547
2     1.364783e+09   1.037547
3     9.203567e+08   1.235174
4     9.203567e+08   1.235174
...            ...        ...
2095  2.033248e+05  

[2100 rows x 2 columns]
                 E          k
0     2.265638e+09   0.000000
1     1.859973e+09   1.090880
2     1.859973e+09   1.090880
3     1.026402e+09   1.298667
4     1.026402e+09   1.298667
...            ...        ...
2095  2.701510e+05  37.661329
2096  2.701510e+05  37.661329
2097  2.396713e+05  37.804930
2098  2.396713e+05  37.804930
2099  2.196087e+05  38.568428

[2100 rows x 2 columns]
                 E          k
0     2.308793e+09   0.000000
1     1.875951e+09   1.093304
2     1.875951e+09   1.093304
3     1.028879e+09   1.301553
4     1.028879e+09   1.301553
...            ...        ...
2095  2.734691e+05  37.745023
2096  2.734691e+05  37.745023
2097  2.407345e+05  37.888943
2098  2.407345e+05  37.888943
2099  2.227119e+05  38.654138

[2100 rows x 2 columns]
                 E          k
0     2.311100e+09   0.000000
1     1.888909e+09   1.095728
2     1.888909e+09   1.095728
3     1.029907e+09   1.304439
4     1.029907e+09   1.304439
...            ...        

                 E          k
0     2.346036e+09   0.000000
1     1.933574e+09   1.158423
2     1.933574e+09   1.158423
3     1.051625e+09   1.379075
4     1.051625e+09   1.379075
...            ...        ...
2095  2.783671e+05  39.993167
2096  2.783671e+05  39.993167
2097  2.558903e+05  40.145659
2098  2.558903e+05  40.145659
2099  2.328542e+05  40.956430

[2100 rows x 2 columns]
                 E          k
0     2.348377e+09   0.000000
1     1.950997e+09   1.160847
2     1.950997e+09   1.160847
3     1.060875e+09   1.381961
4     1.060875e+09   1.381961
...            ...        ...
2095  2.819430e+05  40.076862
2096  2.819430e+05  40.076862
2097  2.568122e+05  40.229672
2098  2.568122e+05  40.229672
2099  2.354397e+05  41.042140

[2100 rows x 2 columns]
                 E          k
0     2.348464e+09   0.000000
1     1.951037e+09   1.163257
2     1.951037e+09   1.163257
3     1.060954e+09   1.384829
4     1.060954e+09   1.384829
...            ...        ...
2095  2.819475e+05  

KeyboardInterrupt: 

In [29]:
data1 = pd.read_csv(r'E:\moreData\08032020\energy_spectrum\00\3000-3001.csv')
data2 = pd.read_csv(r'E:\moreData\08032020\energy_spectrum\00\3100-3101.csv')

In [30]:
count = 0
for data in [data1, data2]:
    if count == 0:
        data.to_csv(r'E:\moreData\08032020\energy_spectrum\data-sum.csv', index=False)
    else:
        data.to_csv(r'E:\moreData\08032020\energy_spectrum\data-sum.csv', index=False, mode='a', header=False)
    count += 1

In [37]:
def summarize_es(folder):
    """
    Average energy spectrum data in each subfolder under folder, then save a summary.csv file under folder, containing all the averaged data.
    
    """
    sfL = next(os.walk(folder))[1]
    count = 0
    for sum_count, sf in enumerate(sfL):
#         clear_output(wait=True)
        print('Summarizing {0}\\{1} ...'.format(folder, sf))
        es_folder = os.path.join(folder, sf) # for varstep, experiment number '00' needs to be included
        l = readdata(es_folder, 'csv')
        data_count = 0
        for num, i in l.iterrows():
            if num == 0:
                data = pd.read_csv(i.Dir)
            else:
                data += pd.read_csv(i.Dir)
            print('{:d}: {:f}'.format(num, data.E.mean()))
            data_count += 1
        data /= data_count + 1
        summary = data.assign(sample=sf)
        
#         if count == 0:
#             summary = data.assign(sample=sf)
#         else:
#             summary = summary.append(data.assign(sample=sf))
        count += 1
        if sum_count == 0:
            summary.to_csv(os.path.join(folder, 'test-summary.csv'), index=False)
        else:
            summary.to_csv(os.path.join(folder, 'test-summary.csv'), index=False, mode='a', header=False)

In [38]:
folders = ['08032020']#, '08042020', '08052020', '08062020']
for f in folders:
    folder = os.path.join(r'E:\moreData', f, 'energy_spectrum')
    summarize_es(folder)

Summarizing E:\moreData\08032020\energy_spectrum\00 ...
0: 5803.637435
1: 12388.711274
2: 18916.931971
3: 25152.935512
4: 31665.934322
5: 38065.459547
6: 44194.292644
7: 50256.689892
8: 56362.609672
9: 62419.705971
10: 68500.829806
11: 74128.303529
12: 80178.966588
13: 86114.081790
14: 92349.160629
15: 98727.096638
16: 105317.247740
17: 111739.827920
18: 118262.945584
19: 124739.709783
20: 131233.399490
21: 138102.030454
22: 144995.665683
23: 151944.049949
24: 158847.952426
25: 165808.496578
26: 172804.080701
27: 179760.700704
28: 186522.852871
29: 193873.472440
30: 201123.314810
31: 208508.499330
32: 216104.742824
33: 223585.538516
34: 230876.817709
35: 238047.123008
36: 245360.000396
37: 252674.013655
38: 259929.623883
39: 267263.683469
40: 274355.208743
41: 281484.595566
42: 288566.777335
43: 295894.712728
44: 303179.395138
45: 310856.788449
46: 318319.654116
47: 325595.430995
48: 332780.913860
49: 339699.011645
50: 346532.044455
51: 353644.289338
52: 360975.523603
53: 368296.877341

415: 248417368.070936
416: 248426569.335240
417: 248435498.719741
418: 248444763.436292
419: 248453919.079759
420: 250975294.109586
421: 250984272.541040
422: 250993409.539642
423: 251002861.001486
424: 251011810.547108
425: 251021008.633238
426: 251030159.769352
427: 253616523.526896
428: 256173116.323509
429: 256182384.607089
430: 256191704.521171
431: 258850069.380001
432: 261485496.994068
433: 264137077.001694
434: 266751233.893271
435: 269233381.991261
436: 269242399.363965
437: 269251305.679784
438: 271688474.112344
439: 271696927.620961
440: 274126029.144133
441: 276723022.498010
442: 276732412.764067
443: 276741298.850900
444: 276750000.235837
445: 279312019.317752
446: 281751299.409231
447: 284366859.418596
448: 286842307.183088
449: 286850951.078049
450: 286860279.652728
451: 286869332.257159
452: 286878403.718395
453: 289551081.371911
454: 289560190.340330
455: 292123784.507276
456: 292132578.169713
457: 292141986.672951
458: 292151099.777891
459: 292159907.264283
460: 29458

167: 95314474491248123482669580288.000000
168: 95314474491248123482669580288.000000
169: 95314474491248123482669580288.000000
170: 95314474491248123482669580288.000000
171: 95314474491248123482669580288.000000
172: 95314474491248123482669580288.000000
173: 95314474491248123482669580288.000000
174: 95314474491248123482669580288.000000
175: 95314474491248123482669580288.000000
176: 95314474491248123482669580288.000000
177: 95314474491248123482669580288.000000
178: 95314474491248123482669580288.000000
179: 95314474491248123482669580288.000000
180: 95314474491248123482669580288.000000
181: 95314474491248123482669580288.000000
182: 95314474491248123482669580288.000000
183: 95314474491248123482669580288.000000
184: 95314474491248123482669580288.000000
185: 95314474491248123482669580288.000000
186: 95314474491248123482669580288.000000
187: 95314474491248123482669580288.000000
188: 95314474491248123482669580288.000000
189: 95314474491248123482669580288.000000
190: 95314474491248123482669580288

399: 95314474491248123482669580288.000000
400: 95314474491248123482669580288.000000
401: 95314474491248123482669580288.000000
402: 95314474491248123482669580288.000000
403: 95314474491248123482669580288.000000
404: 95314474491248123482669580288.000000
405: 95314474491248123482669580288.000000
406: 95314474491248123482669580288.000000
407: 95314474491248123482669580288.000000
408: 95314474491248123482669580288.000000
409: 95314474491248123482669580288.000000
410: 95314474491248123482669580288.000000
411: 95314474491248123482669580288.000000
412: 95314474491248123482669580288.000000
413: 95314474491248123482669580288.000000
414: 95314474491248123482669580288.000000
415: 95314474491248123482669580288.000000
416: 95314474491248123482669580288.000000
417: 95314474491248123482669580288.000000
418: 95314474491248123482669580288.000000
419: 95314474491248123482669580288.000000
420: 95314474491248123482669580288.000000
421: 95314474491248123482669580288.000000
422: 95314474491248123482669580288

0: 14505.120272
1: 27192.682927
2: 52896.865987
3: 65105.453238
4: 66049.261891
5: 66392.121164
6: 67378.304395
7: 68026.373704
8: 68112.416424
9: 73412.060500
10: 80698.684926
11: 81372.198914
12: 82355.761776
13: 82627.296735
14: 82745.868364
15: 83475.222421
16: 84117.037528
17: 84217.017398
18: 90915.266802
19: 92158.792372
20: 95986.347711
21: 96173.838066
22: 96349.726445
23: 96571.675110
24: 96806.602562
25: 97054.971296
26: 97311.710667
27: 97571.924612
28: 97839.087069
29: 98132.881322
30: 98417.376875
31: 98709.732808
32: 98996.626553
33: 99327.924617
34: 99644.132125
35: 99972.160138
36: 100322.122569
37: 100633.560035
38: 101016.815304
39: 101445.521924
40: 101826.213807
41: 102238.262243
42: 102700.359988
43: 103142.758197
44: 103619.413002
45: 104086.951216
46: 104559.765936
47: 105151.588181
48: 105738.404293
49: 106315.476291
50: 106993.719808
51: 107716.813244
52: 108503.571350
53: 109486.009596
54: 110613.461344
55: 114921.502081
56: 119155.176572
57: 123608.883418
58

...

469: 4288371.825388
470: 4313204.426604
471: 4335609.285741
472: 4360262.540112
473: 4360361.650022
474: 4388113.612464
475: 4415293.464617
476: 4444319.998577
477: 4468724.811861
478: 4490826.050886
479: 4514025.700423
480: 4514120.157948
481: 4540620.478632
482: 4567323.148919
483: 4593787.671843
484: 4619095.140773
485: 4619187.741521
486: 4642205.332149
487: 4642292.064692
488: 4642384.350504
489: 4663769.273428
490: 4663868.514183
491: 4693424.107713
492: 4693533.414966
493: 4726465.890995
494: 4755829.812220
495: 4781338.444609
496: 4811124.420340
497: 4836573.064798
498: 4862536.402475
499: 4862639.738418
500: 4889276.895351
501: 4911520.851839
502: 4911606.439962
503: 4934410.944008
504: 4934488.437630
505: 4960177.012973
506: 4985558.845774
507: 5013158.015413
508: 5038983.919914
509: 5039081.408171
510: 5066290.918798
511: 5093336.033186
512: 5118041.016561
513: 5118132.131568
514: 5143567.504859
515: 5170527.903370
516: 5170642.802977
517: 5195028.044177
518: 5220941.922856


148: 383549213431404019041959936.000000
149: 383549213431404019041959936.000000
150: 383549213431404019041959936.000000
151: 383549213431404019041959936.000000
152: 383549213431404019041959936.000000
153: 383549213431404019041959936.000000
154: 383549213431404019041959936.000000
155: 383549213431404019041959936.000000
156: 383549213431404019041959936.000000
157: 383549213431404019041959936.000000
158: 383549213431404019041959936.000000
159: 383549213431404019041959936.000000
160: 383549213431404019041959936.000000
161: 383549213431404019041959936.000000
162: 383549213431404019041959936.000000
163: 383549213431404019041959936.000000
164: 383549213431404019041959936.000000
165: 383549213431404019041959936.000000
166: 383549213431404019041959936.000000
167: 383549213431404019041959936.000000
168: 383549213431404019041959936.000000
169: 383549213431404019041959936.000000
170: 383549213431404019041959936.000000
171: 383549213431404019041959936.000000
172: 383549213431404019041959936.000000


388: 383549213431404019041959936.000000
389: 383549213431404019041959936.000000
390: 383549213431404019041959936.000000
391: 383549213431404019041959936.000000
392: 383549213431404019041959936.000000
393: 383549213431404019041959936.000000
394: 383549213431404019041959936.000000
395: 383549213431404019041959936.000000
396: 383549213431404019041959936.000000
397: 383549213431404019041959936.000000
398: 383549213431404019041959936.000000
399: 383549213431404019041959936.000000
400: 383549213431404019041959936.000000
401: 383549213431404019041959936.000000
402: 383549213431404019041959936.000000
403: 383549213431404019041959936.000000
404: 383549213431404019041959936.000000
405: 383549213431404019041959936.000000
406: 383549213431404019041959936.000000
407: 383549213431404019041959936.000000
408: 383549213431404019041959936.000000
409: 383549213431404019041959936.000000
410: 383549213431404019041959936.000000
411: 383549213431404019041959936.000000
412: 383549213431404019041959936.000000


15: 6784.543276
16: 6883.613465
17: 6970.613756
18: 7041.413954
19: 7114.728978
20: 7202.761673
21: 7294.178294
22: 7377.452858
23: 7478.374642
24: 7567.920332
25: 7654.250102
26: 7741.395550
27: 7837.653719
28: 7925.565572
29: 8011.534541
30: 8089.223298
31: 8179.857693
32: 8261.052488
33: 8347.016214
34: 8445.582813
35: 8531.394334
36: 8617.845777
37: 8708.425330
38: 8793.037805
39: 8873.290567
40: 8954.724724
41: 9045.986954
42: 9125.972803
43: 9215.949025
44: 9294.950813
45: 9371.614089
46: 9458.234463
47: 9537.042220
48: 9624.380231
49: 9709.416308
50: 9785.965677
51: 9859.670612
52: 9953.706894
53: 10044.269559
54: 10123.215202
55: 10208.898936
56: 10290.875313
57: 10402.898538
58: 10489.923757
59: 10586.684907
60: 10690.806777
61: 10784.930323
62: 10884.064011
63: 10976.260994
64: 11052.847856
65: 11132.437769
66: 11203.572195
67: 11297.635984
68: 11397.195054
69: 11474.189296
70: 11569.340864
71: 11646.350419
72: 11724.441438
73: 11809.549349
74: 11891.134044
75: 11979.263907
7

481: 3591283.267765
482: 3617580.973913
483: 3643407.892802
484: 3668652.743010
485: 3693436.912584
486: 3719830.262886
487: 3719916.379279
488: 3743876.497738
489: 3771267.890443
490: 3796981.001436
491: 3818950.161730
492: 3844492.345509
493: 3868670.085671
494: 3894014.883523
495: 3924441.154866
496: 3950640.703435
497: 3977300.624970
498: 3998716.112260
499: 4024066.738955
500: 4047695.213561
501: 4073716.271589
502: 4096146.088658
503: 4123046.221929
504: 4150215.769119
505: 4177465.389628
506: 4202029.779872
507: 4226664.844241
508: 4254691.944426
509: 4276269.854137
510: 4302397.472920
511: 4323490.765062
512: 4349088.216198
513: 4369768.615862
514: 4391676.125254
515: 4414187.219914
516: 4438034.316886
517: 4468568.138989
518: 4468658.954919
519: 4496880.477344
520: 4522587.353417
521: 4522700.314383
522: 4522811.770141
Summarizing E:\moreData\08032020\energy_spectrum\12 ...
0: 12.686824
1: 26.045809
2: 39.230232
3: 52.788962
4: 65.210651
5: 78.326821
6: 3760.972775
7: 3774.751

In [35]:
data1 /= 100

In [39]:
folder = r'E:\moreData\08032020\piv_imseq\00'
l = readdata(folder, 'csv')

In [41]:

l_crop = l[l.index>l.index.max()*0.8]

In [42]:
l_crop

,Name,Dir
1440,2880-2881,E:\moreData\08032020\piv_imseq\00\2880-2881.csv
1441,2882-2883,E:\moreData\08032020\piv_imseq\00\2882-2883.csv
1442,2884-2885,E:\moreData\08032020\piv_imseq\00\2884-2885.csv
1443,2886-2887,E:\moreData\08032020\piv_imseq\00\2886-2887.csv
1444,2888-2889,E:\moreData\08032020\piv_imseq\00\2888-2889.csv
...,...,...
1795,3590-3591,E:\moreData\08032020\piv_imseq\00\3590-3591.csv
1796,3592-3593,E:\moreData\08032020\piv_imseq\00\3592-3593.csv
1797,3594-3595,E:\moreData\08032020\piv_imseq\00\3594-3595.csv
1798,3596-3597,E:\moreData\08032020\piv_imseq\00\3596-3597.csv
